# Notebook to Clean Data

2019 - 01 - 21 updated 2019-04-30
Notebook to clean final osm cities and towns
1. merge files
2. drop duplicates 


In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import json
from shapely.geometry import Point

In [2]:
import os
os.getcwd()

'/tana-crunch/cascade/projects/Pop-ERL/notebooks/jupyter/ERL19'

In [3]:
# File Paths

ERLv2_data = '../../../temp_data/ERL19v2/'
temp_data = '../../../temp_data/ERL19/'

### Merge OSM Files

In [ ]:
def load_points (file):
    """ This function loads a csv 
    of points and turns it into shapely points"""
    df = pd.read_csv(file)

    # creating a geometry column 
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

    # Coordinate reference system : WGS84
    crs = {'init': 'epsg:4326'}

    # Creating a Geographic data frame 
    point_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
    
    return point_gdf

In [ ]:
# Load Files
town = load_points(v2_data+'20190114_osm_africa_towns.csv')
city = load_points(v2_data+'20190114_osm_africa_cities.csv')

townSS = load_points(v2_data+'20190221_osm_S_Sudan_towns.csv')
citySS = load_points(v2_data+'20190221_osm_S_Sudan_cities.csv')

townDRC = load_points(v2_data+'20190430_osm_DRC_towns.csv')
cityDRC = load_points(v2_data+'20190430_osm_DRC_cities.csv')

In [ ]:
# add column with osm type

town['osm_type'] = 'town'
townDRC['osm_type'] = 'town'
townSS['osm_type'] = 'town'

city['osm_type'] = 'city'
cityDRC['osm_type'] = 'city'
citySS['osm_type'] = 'city'

In [ ]:
cityDRC.head()

In [ ]:
# merge

towns = pd.concat([town, townDRC, townSS, cityDRC, citySS], sort = False) # col name of cityDRC and citySS is town, not city

towns.rename(columns={'town':'osm_name'}, inplace=True)
city.rename(columns={'city':'osm_name'}, inplace=True)

out = pd.concat([towns, city], sort = False)


In [ ]:
out[1000:1100]

In [ ]:
out.shape

In [ ]:
out.to_file(v2_data+"20190430_osm_All.shp", driver = "ESRI Shapefile")

### Split 1500c300 Polygons

In [ ]:
# Load

GHS2000 = gpd.read_file(temp_data+'GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300.shp')


In [ ]:
GHS2000.head()

In [ ]:
# ADD FIDS if needed

FID = list(range(len(GHS2000)))
GHS2000['DN'] = FID
GHS2000.columns.values[0] = "FID"
GHS2000.head()

In [ ]:
len(GHS2000)

In [ ]:
GHS2000_A = GHS2000[0:15000]
len(GHS2000_A)

In [ ]:
GHS2000_B = GHS2000[15000:30000]
len(GHS2000_B)

In [ ]:
GHS2000_C = GHS2000[30000:]
len(GHS2000_C)

In [ ]:
len(GHS2000_A)+len(GHS2000_B)+len(GHS2000_C)

In [ ]:
GHS2000_A.tail()

In [ ]:
GHS2000_C.head()

In [ ]:
GHS2000_A.to_file(temp_data+'GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_A.shp')
GHS2000_B.to_file(temp_data+'GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_B.shp')
GHS2000_C.to_file(temp_data+'GHS_POP_GPW42000_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_C.shp')

### Merge PolyPoints 
- Merge polypoints outputs & save
- Find FIDs that overlap boarders

In [ ]:
# file paths

poly_A = gpd.read_file(ERLv2_data+'LS15_w001001_Clip_1500c300_A_polypoints.shp')
poly_B = gpd.read_file(ERLv2_data+'LS15_w001001_Clip_1500c300_B_polypoints.shp')
#poly_C = gpd.read_file(ERLv2_data+'GHS_POP_GPW42015_GLOBE_R2015A_54009_1k_v1_0_Clip_1500c300_C_polypoints.shp')

out_data = 'LS2015'

In [ ]:
poly = pd.concat([poly_A, poly_B], ignore_index = True) #poly_C

In [ ]:
len(poly)

In [ ]:
# Save out all Polygons
poly.to_file(ERLv2_data+out_data+'_polypoints_ALL.shp')

### Find Country Overlap

In [ ]:
# open files
out_data = 'WP2015'

#file = out_data+'_polypoints_ALL.shp'
file = 'WP2015_1500c300_polypoints.shp'

poly_all = gpd.read_file(ERLv2_data+file)

In [ ]:
step1 = poly_all
step1['dup'] = poly_all['FID'].astype(str) + poly_all['country'] # Make new col with FID-Country
step2 = step1.drop_duplicates('dup', keep='first') # drop country-FID dups, keep first though
step3 = step2[step2.duplicated(subset=['FID'], keep=False)] # keep all with duplicated FIDS, drop unique due to no country overlap
step4 = step3.drop_duplicates('FID', keep='first') # drop country-FID dups remaining duplicates
step4

In [ ]:
# Drop Western Sahara / Morocco and South Sudan / Sudan
step4 = step4[step4['country'] != 'Sudan']

print(len(step4))

step4 = step4[step4['country'] != 'Morocco']
print(len(step4))

# out = out[out.duplicated(subset=['FID'], keep=False)] # Keep all duplicated FIDS but remove any left overs


In [ ]:
step4.to_file(ERLv2_data+out_data+'_polypoints_countryoverlap.shp')


#### Old Code

In [ ]:
# FIDS with GHS2015_ALL to check

testA = poly[poly['FID'] == 27492] # double boarder
testB = poly[poly['FID'] == 31036] #lagos
testC = poly[poly['FID'] == 187] # random city only in Algeria 
testD = poly[poly['FID'] == 28] # large city in S africa
testE = poly[poly['FID'] == 18] # single city

test = pd.concat([testA, testB, testC, testD, testE])
test

In [ ]:
# ## Code to find polygons that overlap boarders

# test = poly[poly.duplicated(subset=['FID'], keep=False)] # Keep all dup FIDS
# test['dup'] = test['FID'].astype(str) + test['country'] # Make new col with FID-Country
# test.head()

# out = test.drop_duplicates('dup', keep=False) # Drop all doubles FID-Country
# print(len(out))

# # out = out.drop_duplicates('FID', keep='first') # Drop all left double FIDs
# # print(len(out))

In [ ]:
# Fake Data

fid = [1,2,3,2,2,3]
country = ['A','B','C','A','B','A']
pop = [10,11,12,11,11,12]

df = pd.DataFrame()
df['FID'] = fid
df['country'] = country
df['pop'] = pop

df


In [ ]:
step1A = poly[poly['FID'] == 31036] #lagos
step1B = poly[poly['FID'] == 187] # random city only in Algeria 

step1 = pd.concat([step1A, step1B])

In [ ]:
# Make new col with FID-Country

step1['dup'] = step1['FID'].astype(str) + step1['country'] # Make new col with FID-Country
step1

In [ ]:
# drop duplicated country-FID, but keep first

step2 = step1.drop_duplicates('dup', keep='first')
step2

In [ ]:
# keep all duplicated FIDS, remove singles

step3 = step2[step2.duplicated(subset=['FID'], keep=False)] 
step3

In [ ]:
# Drop remaining duplicated FIDS 

step4 = step3.drop_duplicates('FID', keep='first')
step4